In [1]:
import autograd
from autograd import numpy as np
import VariationalBayes as vb

In [7]:
params = vb.ModelParamsDict('Full')
params.push_param(vb.VectorParam('a', size=5))
params.push_param(vb.ScalarParam('b'))
params.push_param(vb.PosDefMatrixParam('c', size=3))
print(params)

Full:
	a:
[ 0.  0.  0.  0.  0.]
	b: 0.0
	c:
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


In [8]:
sub_params = vb.ModelParamsDict('Sub')
sub_params.push_param(params['a'])
sub_params['a'].set_free(np.random.random(sub_params['a'].free_size()))
print(params['a'])
print(sub_params['a'])

a:
[ 0.66490073  0.46797731  0.30266936  0.5819356   0.12881259]
a:
[ 0.66490073  0.46797731  0.30266936  0.5819356   0.12881259]


In [9]:
sub_params = vb.ModelParamsDict('Sub')
sub_params.push_param(params['b'])
sub_params['b'].set_free(np.random.random(sub_params['b'].free_size()))
print(params['b'])
print(sub_params['b'])

b: [ 0.41888219]
b: [ 0.41888219]


In [10]:
sub_params = vb.ModelParamsDict('Sub')
sub_params.push_param(params['c'])
sub_params['c'].set_free(np.random.random(sub_params['c'].free_size()))
print(params['c'])
print(sub_params['c'])

c:
[[ 2.14834943  0.47125169  1.22076325]
 [ 0.47125169  1.81840264  1.10251585]
 [ 1.22076325  1.10251585  3.39606233]]
c:
[[ 2.14834943  0.47125169  1.22076325]
 [ 0.47125169  1.81840264  1.10251585]
 [ 1.22076325  1.10251585  3.39606233]]


In [11]:
print(params)

Full:
	a:
[ 0.66490073  0.46797731  0.30266936  0.5819356   0.12881259]
	b: [ 0.41888219]
	c:
[[ 2.14834943  0.47125169  1.22076325]
 [ 0.47125169  1.81840264  1.10251585]
 [ 1.22076325  1.10251585  3.39606233]]


In [39]:
from VariationalBayes import ArrayParam


ap = vb.ArrayParam('full', shape=(6, 8))
ap.set_free(np.random.random(ap.free_size()))
ap_shape = ap.shape()




In [41]:
ap_slice = (slice(1, 3), slice(2, 5))
print(ap.get())
print('\n')
print(ap.get()[ap_slice])


[[ 0.01699038  0.33888272  0.9616943   0.54846651  0.86943449  0.53276709
   0.70249896  0.40713905]
 [ 0.47774689  0.23713409  0.11504061  0.24598823  0.4758053   0.08409119
   0.57398639  0.07543381]
 [ 0.70387704  0.92138867  0.18190259  0.20737976  0.37043285  0.05306023
   0.26600189  0.40355764]
 [ 0.45673711  0.22750412  0.58126048  0.33795698  0.45951934  0.59032289
   0.38486094  0.78273733]
 [ 0.03186865  0.71263906  0.13319375  0.52827641  0.5207389   0.32408118
   0.24011857  0.77501691]
 [ 0.62172062  0.40669053  0.24492825  0.4240796   0.858818    0.44201267
   0.34054959  0.15988795]]


[[ 0.11504061  0.24598823  0.4758053 ]
 [ 0.18190259  0.20737976  0.37043285]]


In [58]:
foo = slice(2, 3, 1)
assert foo.step == 1
ap_slice_shape = tuple([ axis_slice.stop - axis_slice.start for axis_slice in ap_slice])
print(ap_slice_shape)

(2, 3)


In [60]:
# Maybe you can get differentiable array assignments like this.

mask = np.ones(ap_shape)
mask[ap_slice] = 0.

new_vals = np.random.random(ap_slice_shape)
print(new_vals)


[[ 0.49629572  0.12351255  0.89077062]
 [ 0.64074006  0.75747942  0.31999968]]


In [13]:

class SubArrayParam(object):
    def __init__(self, name='', full_array_param=ArrayParam(), ap_slice=None):
        self.name = name
        self.full_array_param = full_array_param
        self.__full_shape = full_array_param.shape()
        if ap_slice is None:
            self.ap_slice = self.get_full_slice()
        else:
            self.ap_slice = ap_slice
        
    def get_full_slice(self):
        tuple([ slice(0, axis_size) for axis_size in self.__full_shape ])
        
    def __str__(self):
        return self.name + ':\n' + str(self.__val)
    def names(self):
        return self.name
    def dictval(self):
        return self.__val.tolist()

    def set(self, val):
        # TODO: check for bounds and shapes
        
        # ... Got to this point and realized that setting this in a differentiable
        # manner will be tricky.
        self.__val = val
    def get(self):
        return self.full_array_param.get()[self.ap_slice]

    def set_free(self, free_val):
        if free_val.size != self.free_size():
            error_string = \
                'Wrong size for array {}.  Expected {}, got {}'.format(
                    self.name, str(self.free_size()), str(free_val.size))
            raise ValueError(error_string)
        self.set(constrain(free_val, self.__lb, self.__ub).reshape(self.__shape))
    def get_free(self):
        return unconstrain_array(self.__val, self.__lb, self.__ub)
    def free_to_vector(self, free_val):
        self.set_free(free_val)
        return self.get_vector()
    def free_to_vector_jac(self, free_val):
        rows_indices = np.array(range(self.vector_size()))
        grads = [ constrain_scalar_jac(free_val[vec_ind], self.__lb, self.__ub) \
                  for vec_ind in range(self.vector_size()) ]
        return coo_matrix((grads,
                          (rows_indices, rows_indices)),
                          (self.vector_size(), self.free_size()))
    def free_to_vector_hess(self, free_val):
        def get_ind_hess(vec_ind):
            hess = constrain_scalar_hess(free_val[vec_ind], self.__lb, self.__ub)
            return coo_matrix(([ hess ],
                               ([vec_ind], [vec_ind])),
                               (self.free_size(), self.vector_size()))
        return np.array([ get_ind_hess(vec_ind)
                          for vec_ind in range(self.vector_size()) ])

    def set_vector(self, val):
        if val.size != self.vector_size():
            error_string = \
                'Wrong size for array {}.  Expected {}, got {}'.format(
                    self.name, str(self.vector_size()), str(val.size))
            raise ValueError(error_string)
        self.set(val.reshape(self.__shape))
    def get_vector(self):
        return self.__val.flatten()

    def shape(self):
        return self.__shape
    def free_size(self):
        return int(np.product(self.__shape))
    def vector_size(self):
        return int(np.product(self.__shape))
